najmanji broj klastera za najvecim siluet skorom
siluet skor za svaki klaster i onda mean

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sb
import multiprocessing as mp
import os
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans # type: ignore

In [6]:
data = pd.read_csv('smaller_dataset.csv', index_col=0)

In [ ]:
data.reset_index(inplace=True, drop=True)
data.head()

In [8]:
data['CellID'] = data['CellID'].astype(np.uint16)
data['ExonCount'] = data['ExonCount'].astype(np.uint8)
data['MIDCount'] = data['MIDCount'].astype(np.uint8)
data['x'] = data['x'].astype(np.uint16)
data['y'] = data['y'].astype(np.uint16)

In [9]:
unique_cells = data['CellID'].unique()
unique_genes = data['geneID'].unique()

In [10]:
xmin, ymin = data['x'].min(), data['y'].min()
data['x'], data['y'] = data['x'] - xmin, data['y'] - ymin
xmax, ymax = data['x'].max(), data['y'].max()
xmax += 1
ymax += 1

In [ ]:
def generate_picture() -> None:
    __data = data[['x', 'y']]
    picture = np.zeros(shape=(xmax, ymax), dtype=np.uint8)

    for row in range(__data.shape[0]):
        x, y = __data.iloc[row]
        picture[x][y] = 1

    plt.figure(figsize=(xmax/100, ymax/100))
    plt.imshow(picture, cmap='gray')
    plt.axis('off')
    plt.show()
    plt.imsave('cells.tiff', picture, cmap='gray')

    return picture

picture = generate_picture()

In [ ]:
def midcount_filling() -> int:
    matrix = np.zeros((len(unique_cells), len(unique_genes)), dtype=np.uint8)
    cell_indices = pd.factorize(data['CellID'])[0] 
    gene_indices = pd.factorize(data['geneID'])[0]

    matrix_cell_indices = np.array(cell_indices, dtype=np.uint16)
    matrix_gene_indices = np.array(gene_indices, dtype=np.uint16)
    matrix_midcount = np.array(data['MIDCount'].values, dtype=np.uint8)

    matrix[matrix_cell_indices, matrix_gene_indices] = matrix_midcount

    unloged_matrix = pd.DataFrame(matrix, index=unique_cells, columns=unique_genes, dtype=np.uint8)
    unloged_matrix.to_csv('unloged_matrix.csv', index=False)

    unloged_matrix.info()
    return unloged_matrix.last_valid_index()

unloged_matrix_number_of_rows = midcount_filling()

In [ ]:
half_rows = unloged_matrix_number_of_rows // 2
second_half_rows = unloged_matrix_number_of_rows - half_rows

def loging_matrix(data_chunk, mode) -> None:
    matrix = np.array(data_chunk.values, dtype=np.float32)

    matrix[matrix == 0] = -1
    matrix = np.where(matrix == -1, -1, np.log(matrix))

    loged_matrix = pd.DataFrame(matrix, index=data_chunk.index, columns=data_chunk.columns, dtype=np.float32)
    loged_matrix.to_csv('loged_matrix.csv', mode=mode, header=(mode == 'w'), index=False)

def first_part() -> None:
    first_half = pd.read_csv('unloged_matrix.csv', nrows=half_rows)
    loging_matrix(first_half, mode='w')

def second_part() -> None:
    second_half = pd.read_csv('unloged_matrix.csv', skiprows=half_rows, nrows=second_half_rows)
    loging_matrix(second_half, mode='a')

first_part()
second_part()

In [4]:
def fetch(file_name : str) -> np.ndarray:
    tmp = pd.read_csv(file_name, index_col=0)
    return np.array(tmp.values)

def clustering() -> list:
    data = fetch('unloged_matrix.csv')
    model = KMeans(n_clusters=20) ###
    model.fit(data)
    return model.labels_

labels = clustering()

In [ ]:
def generate_clustered_picture() -> None:
    __data = data[['x', 'y', 'CellID']]
    __labels = pd.DataFrame(labels, index=unique_cells)
    picture = np.zeros(shape=(xmax, ymax), dtype=np.uint8)

    for row in range(__data.shape[0]):
        x, y, cellid = __data.iloc[row]
        picture[x][y] = __labels.loc[cellid][0]

    cmap = plt.get_cmap('tab20', 20) ###
    tmp = list(cmap.colors)
    tmp[0] = (0, 0, 0) 
    cmap = mcolors.ListedColormap(tmp)
    
    plt.figure(figsize=(xmax/100, ymax/100))
    plt.imshow(picture, cmap=cmap)
    plt.axis('off')
    plt.colorbar(ticks=np.arange(20)) ###
    plt.show()
    plt.imsave('clustered_cells.tiff', picture, cmap=cmap)

    return picture

clustered_picture = generate_clustered_picture()